In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [4]:
import os
os.environ["Java_Home"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["Spark_home"]="/content/spark-3.0.0-bin-hadoop3.2"

In [5]:
!pip install -q findspark

In [6]:
!pip install pyspark==3.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.7 MB 24 kB/s 
     |████████████████████████████████| 198 kB 21.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=2bd4f7eee9ab2273bc86b6bb4d81bea469b084cb5dd7442612ea556d1e576119
  Stored in directory: /root/.cache/pip/wheels/4e/c5/36/aef1bb711963a619063119cc032176106827a129c0be20e301
Successfully built pyspark


In [7]:
import findspark
findspark.init()

In [8]:
import pyspark
findspark.init("spark-3.0.0-bin-hadoop3.2")

In [9]:
from pandas import isnull
import numpy as np
import pandas as pd
import pandas_profiling

import os
import sys

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

import pyspark.sql.functions as func
from pyspark.sql.functions import col,avg

import seaborn as sns
from pyspark.sql.types import *


In [10]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
!unzip -x /content/airbnb_nyc_clean.csv.zip

Archive:  /content/airbnb_nyc_clean.csv.zip
  inflating: airbnb_nyc_clean.csv    


In [12]:
MyData='airbnb_nyc_clean.csv'
My_df=spark.read.csv(MyData,inferSchema=True,header=True)

In [13]:
My_df.show()

+-------+--------------------+-----------+----------------------+---------+-------------------+------------------+--------+---------+----------------+-------------------+---------------+-----------------+------+------------------+--------------+-----------------+-----------+-----------------+------------------+------------------------------+----------------+--------------------+
|     id|                name|    host_id|host_identity_verified|host_name|neighbourhood_group|     neighbourhood|     lat|     long|instant_bookable|cancellation_policy|      room_type|construction_year| price|       service_fee|minimum_nights|number_of_reviews|last_review|reviews_per_month|review_rate_number|calculated_host_listings_count|availability_365|         house_rules|
+-------+--------------------+-----------+----------------------+---------+-------------------+------------------+--------+---------+----------------+-------------------+---------------+-----------------+------+------------------+------

In [14]:
My_df.columns

['id',
 'name',
 'host_id',
 'host_identity_verified',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'lat',
 'long',
 'instant_bookable',
 'cancellation_policy',
 'room_type',
 'construction_year',
 'price',
 'service_fee',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'review_rate_number',
 'calculated_host_listings_count',
 'availability_365',
 'house_rules']

In [15]:
My_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_identity_verified: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- instant_bookable: string (nullable = true)
 |-- cancellation_policy: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- construction_year: string (nullable = true)
 |-- price: string (nullable = true)
 |-- service_fee: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- review_rate_number: string (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- availability_365: string (nullable = true)
 |-- hou

Printing the number of rows

In [16]:
print("No.of.rows :", My_df.count())

No.of.rows : 69587


Checking Null values for each attribute

In [17]:
for col in My_df.columns:
  print(col, ":", My_df.where(My_df[col].isNull()).count())

id : 0
name : 27
host_id : 283
host_identity_verified : 283
host_name : 283
neighbourhood_group : 283
neighbourhood : 283
lat : 283
long : 283
instant_bookable : 283
cancellation_policy : 283
room_type : 283
construction_year : 283
price : 283
service_fee : 283
minimum_nights : 283
number_of_reviews : 283
last_review : 283
reviews_per_month : 283
review_rate_number : 283
calculated_host_listings_count : 283
availability_365 : 283
house_rules : 518


#Cleaning of Dataset : 

In [18]:
#Cleaning of Dataset
My_df= My_df.na.drop()

In [19]:
for col in My_df.columns:
  print(col, ":", My_df.where(My_df[col].isNull()).count())

id : 0
name : 0
host_id : 0
host_identity_verified : 0
host_name : 0
neighbourhood_group : 0
neighbourhood : 0
lat : 0
long : 0
instant_bookable : 0
cancellation_policy : 0
room_type : 0
construction_year : 0
price : 0
service_fee : 0
minimum_nights : 0
number_of_reviews : 0
last_review : 0
reviews_per_month : 0
review_rate_number : 0
calculated_host_listings_count : 0
availability_365 : 0
house_rules : 0


In [20]:
My_df.show()

+-------+--------------------+-----------+----------------------+---------+-------------------+------------------+--------+---------+----------------+-------------------+---------------+-----------------+------+------------------+--------------+-----------------+-----------+-----------------+------------------+------------------------------+----------------+--------------------+
|     id|                name|    host_id|host_identity_verified|host_name|neighbourhood_group|     neighbourhood|     lat|     long|instant_bookable|cancellation_policy|      room_type|construction_year| price|       service_fee|minimum_nights|number_of_reviews|last_review|reviews_per_month|review_rate_number|calculated_host_listings_count|availability_365|         house_rules|
+-------+--------------------+-----------+----------------------+---------+-------------------+------------------+--------+---------+----------------+-------------------+---------------+-----------------+------+------------------+------

Dropping few **columns**

In [21]:
My_df= My_df.drop("id","host_name","host_identity_verified","instant_bookable","cancellation_policy","construction_year","service_fee","last_review","review_rate_number","house_rules")


In [22]:
My_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- availability_365: string (nullable = true)



In [23]:
My_df.select("name").distinct().count()

61028

In [24]:
My_df.select("name").show()

+--------------------+
|                name|
+--------------------+
|Clean & quiet apt...|
|Skylit Midtown Ca...|
|THE VILLAGE OF HA...|
|               blank|
|Entire Apt: Spaci...|
|Large Cozy 1 BR A...|
|     BlissArtsSpace!|
|     BlissArtsSpace!|
|Large Furnished R...|
|Cozy Clean Guest ...|
|Cute & Cozy Lower...|
|Beautiful 1br on ...|
|Central Manhattan...|
|Lovely Room 1, Ga...|
|Wonderful Guest B...|
|West Village Nest...|
|Only 2 stops to M...|
|Perfect for Your ...|
|     Chelsea Perfect|
|Hip Historic Brow...|
+--------------------+
only showing top 20 rows



Neighbourhoodgroup-Count

In [25]:
My_df.select("neighbourhood_group").distinct().count()

23

In [26]:
neighbourhood_group_count= My_df.select("neighbourhood_group").where(My_df.neighbourhood_group.isin("Queens","Brooklyn","Staten Island","Manhattan","Bronx") ).groupby("neighbourhood_group").count()

neighbourhood_group_count.show()

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|             Queens| 9199|
|           Brooklyn|27818|
|      Staten Island|  646|
|          Manhattan|29493|
|              Bronx| 1892|
+-------------------+-----+



Neighbourhood_Group - PriceAverage

In [27]:
neighbourhood_group_PriceAvg= My_df.select("neighbourhood_group","price").where(My_df.neighbourhood_group.isin("Queens","Brooklyn","Staten Island","Manhattan","Bronx") ).groupby("neighbourhood_group").agg(func.mean("price")).alias("AvgPrice")
neighbourhood_group_PriceAvg.show()

+-------------------+-----------------+
|neighbourhood_group|       avg(price)|
+-------------------+-----------------+
|             Queens|627.6383618124546|
|           Brooklyn|626.3778841892898|
|      Staten Island|619.7232325236098|
|          Manhattan|622.0589870063856|
|              Bronx| 629.893843811093|
+-------------------+-----------------+



Neighbourhood - Count

In [28]:
My_df.select("neighbourhood").distinct().count()

251

In [29]:
neighbourhood_count=My_df.groupby("neighbourhood").count().orderBy('count',ascending=False)
neighbourhood_count.show()

+------------------+-----+
|     neighbourhood|count|
+------------------+-----+
|Bedford-Stuyvesant| 5317|
|      Williamsburg| 5070|
|            Harlem| 3634|
|          Bushwick| 3264|
|    Hell's Kitchen| 2807|
|   Upper West Side| 2596|
|   Upper East Side| 2453|
|           Midtown| 2313|
|      East Village| 2279|
|     Crown Heights| 2166|
|           Chelsea| 1555|
|       East Harlem| 1544|
|        Greenpoint| 1438|
|   Lower East Side| 1365|
|           Astoria| 1269|
|Washington Heights| 1159|
|Financial District| 1023|
|      West Village|  966|
|          Flatbush|  850|
|     East Flatbush|  828|
+------------------+-----+
only showing top 20 rows



Neighbourhood - PriceAvg

In [30]:
neighbourhood_PriceAvg= My_df.groupBy("neighbourhood").agg(func.mean("price")).alias("AvgPrice")
neighbourhood_PriceAvg.show()

+------------------+-----------------+
|     neighbourhood|       avg(price)|
+------------------+-----------------+
|            Corona| 613.042735042735|
|      Henry Street|            613.0|
|      Richmondtown|           413.75|
|      Prince's Bay|501.1666666666667|
|       Westerleigh|435.6666666666667|
|        Mill Basin|           736.25|
|      Civic Center|633.9516129032259|
|        Douglaston|            531.0|
|        Mount Hope|685.4418604651163|
|            Queens|           2007.5|
|       Marble Hill|           524.25|
|         Rego Park| 612.549590192846|
|     Dyker Heights|            608.6|
| Kew Gardens Hills|673.4090909090909|
|      Dongan Hills|668.3636363636364|
|Financial District|630.7912728642893|
|       Bay Terrace|            635.0|
|           Midtown|608.1285606142845|
|     Fulton Street|            938.0|
|   Jackson Heights| 627.380272253864|
+------------------+-----------------+
only showing top 20 rows



Combining Neighbourhood and Neighbourhood group

In [31]:
neighbourhood_neighbourhoodGroup=My_df.select(My_df["neighbourhood_group"],My_df["neighbourhood"],My_df["price"]).orderBy('neighbourhood_group',ascending=True)
neighbourhood_neighbourhoodGroup.show()

+-------------------+------------------+---------+
|neighbourhood_group|     neighbourhood|    price|
+-------------------+------------------+---------+
|                  2|                3"|-73.94134|
|              Bronx|         Concourse|   1075.0|
|              Bronx|         Soundview|    934.0|
|              Bronx|    Williamsbridge|    137.0|
|              Bronx|        Mott Haven|    978.0|
|              Bronx|         Concourse|    314.0|
|              Bronx|      Clason Point|    272.0|
|              Bronx|        Mott Haven|    630.0|
|              Bronx|         Concourse|   1180.0|
|              Bronx|       Morris Park|    358.0|
|              Bronx|       Morris Park|    235.0|
|              Bronx|           Belmont|    682.0|
|              Bronx|         Concourse|    737.0|
|              Bronx|        Mott Haven|    268.0|
|              Bronx|       Port Morris|    265.0|
|              Bronx|Westchester Square|    681.0|
|              Bronx|       Por

HostID - Count

In [32]:
My_df.select("host_id").distinct().count()

69065

In [34]:
host_id_count=My_df.groupBy("host_id").count().orderBy('count',ascending=False)
host_id_count.show()

+-----------+-----+
|    host_id|count|
+-----------+-----+
|   verified|    3|
|29531702698|    2|
|       cozy|    2|
|68958967655|    1|
| 4007964313|    1|
|43945171386|    1|
|68778778568|    1|
|85791349998|    1|
|91819937803|    1|
| 4565576303|    1|
|64602961286|    1|
|11609383313|    1|
|75082250321|    1|
|19470616481|    1|
|66739732370|    1|
|38544479718|    1|
|69103209663|    1|
|16278582059|    1|
|83146353759|    1|
|69474619521|    1|
+-----------+-----+
only showing top 20 rows



RoomType - Count

In [35]:
room_typeCount=My_df.where(My_df.room_type.isin("Private room","Shared room","Entire home/apt")).groupBy(My_df["room_type"]).count().orderBy('count',ascending=True)
room_typeCount.show()

+---------------+-----+
|      room_type|count|
+---------------+-----+
|    Shared room| 1468|
|   Private room|30364|
|Entire home/apt|37101|
+---------------+-----+



Room - AvgPrice

In [36]:
room_AvgPrice=My_df.agg(func.mean('price').alias('Avgprice'))
room_AvgPrice.show()

+-----------------+
|         Avgprice|
+-----------------+
|624.9150989782044|
+-----------------+



RoomType - AvgPrice

In [37]:
room_typeAvgPrice=My_df.where(My_df.room_type.isin("Private room","Shared room","Entire home/apt")).groupBy("room_type").agg(func.mean('price').alias('Avgprice'))
room_typeAvgPrice.show()

+---------------+-----------------+
|      room_type|         Avgprice|
+---------------+-----------------+
|    Shared room|624.1484254500489|
|Entire home/apt|624.4195887240523|
|   Private room|624.9912820234102|
+---------------+-----------------+



NeighbourhoodGroup - RoomType - Count

In [38]:
neighGroup_roomType=My_df.select(My_df["neighbourhood_group"],My_df["room_type"]).where(My_df.room_type.isin("Private room","Shared room","Entire home/apt")).groupBy("neighbourhood_group","room_type").count().orderBy("neighbourhood_group",ascending=True)
neighGroup_roomType.show()

+-------------------+---------------+-----+
|neighbourhood_group|      room_type|count|
+-------------------+---------------+-----+
|              Bronx|    Shared room|   82|
|              Bronx|Entire home/apt|  754|
|              Bronx|   Private room| 1056|
|           Brooklyn|   Private room|13183|
|           Brooklyn|Entire home/apt|14103|
|           Brooklyn|    Shared room|  525|
|          Manhattan|    Shared room|  600|
|          Manhattan|   Private room|10585|
|          Manhattan|Entire home/apt|18208|
|             Queens|    Shared room|  250|
|             Queens|   Private room| 5244|
|             Queens|Entire home/apt| 3697|
|      Staten Island|   Private room|  296|
|      Staten Island|    Shared room|   11|
|      Staten Island|Entire home/apt|  339|
+-------------------+---------------+-----+



NeighbourhoodGroup - RoomType - AvgPrice

In [39]:
neighGroup_roomType_AvgPrice=My_df.select(My_df["neighbourhood_group"],My_df["room_type"],My_df["price"]).where(My_df.room_type.isin("Private room","Shared room","Entire home/apt")).groupBy("neighbourhood_group","room_type").agg(func.mean('price').alias('Avgprice')).orderBy("neighbourhood_group",ascending=True)
neighGroup_roomType_AvgPrice.show()


+-------------------+---------------+-----------------+
|neighbourhood_group|      room_type|         Avgprice|
+-------------------+---------------+-----------------+
|              Bronx|    Shared room| 574.719512195122|
|              Bronx|   Private room|635.6921820836515|
|              Bronx|Entire home/apt|627.7734856899893|
|           Brooklyn|   Private room|626.2824385256538|
|           Brooklyn|Entire home/apt| 626.531512085622|
|           Brooklyn|    Shared room|623.5098673341331|
|          Manhattan|    Shared room|625.8270136837532|
|          Manhattan|   Private room|621.9314844074574|
|          Manhattan|Entire home/apt|621.6804554655175|
|             Queens|    Shared room|          634.968|
|             Queens|Entire home/apt|628.5089447960758|
|             Queens|   Private room|626.9717241421965|
|      Staten Island|   Private room|603.6434869600268|
|      Staten Island|    Shared room|685.6363636363636|
|      Staten Island|Entire home/apt|631.6245901

Neighbourhood - RoomType - Count

In [40]:
neigh_roomType=My_df.select(My_df["neighbourhood"],My_df["room_type"]).where(My_df.room_type.isin("Private room","Shared room","Entire home/apt")).groupBy("neighbourhood","room_type").count().orderBy("neighbourhood",ascending=True)
neigh_roomType.show()

+-----------------+---------------+-----+
|    neighbourhood|      room_type|count|
+-----------------+---------------+-----+
|      21st Street|Entire home/apt|    1|
|       5th Avenue|   Private room|    1|
|         Allerton|   Private room|   38|
|         Allerton|Entire home/apt|   34|
|    Arden Heights|Entire home/apt|    5|
|    Arden Heights|   Private room|    1|
|         Arrochar|Entire home/apt|   19|
|         Arrochar|   Private room|   12|
|          Arverne|    Shared room|    4|
|          Arverne|Entire home/apt|  102|
|          Arverne|   Private room|   46|
|          Astoria|Entire home/apt|  523|
|          Astoria|    Shared room|   28|
|          Astoria|   Private room|  718|
|         Avenue A|   Private room|    1|
|       Bath Beach|   Private room|   16|
|       Bath Beach|Entire home/apt|   16|
|Battery Park City|   Private room|   23|
|Battery Park City|Entire home/apt|   58|
|Battery Park City|    Shared room|    3|
+-----------------+---------------

Neighbourhood - RoomType - AvgPrice

In [41]:
neigh_roomType_AvgPrice=My_df.select(My_df["neighbourhood"],My_df["room_type"],My_df["price"]).where(My_df.room_type.isin("Private room","Shared room","Entire home/apt")).groupBy("neighbourhood","room_type").agg(func.mean('price').alias('Avgprice')).orderBy("neighbourhood",ascending=True)
neigh_roomType_AvgPrice.show()

+-----------------+---------------+-----------------+
|    neighbourhood|      room_type|         Avgprice|
+-----------------+---------------+-----------------+
|      21st Street|Entire home/apt|           1111.0|
|       5th Avenue|   Private room|            793.0|
|         Allerton|   Private room|699.8088439495785|
|         Allerton|Entire home/apt|546.4411764705883|
|    Arden Heights|Entire home/apt|            787.8|
|    Arden Heights|   Private room|            604.0|
|         Arrochar|   Private room|495.9166666666667|
|         Arrochar|Entire home/apt| 675.880845793894|
|          Arverne|    Shared room|           988.25|
|          Arverne|Entire home/apt|            651.0|
|          Arverne|   Private room|653.7391304347826|
|          Astoria|    Shared room|617.4285714285714|
|          Astoria|Entire home/apt|621.5190671896137|
|          Astoria|   Private room|641.4626855295988|
|         Avenue A|   Private room|            486.0|
|       Bath Beach|Entire ho

Minimun Nights - Count

In [42]:
min_night_count=My_df.groupBy("minimum_nights").count().orderBy('count',ascending=False)
min_night_count.show()

+--------------+-----+
|minimum_nights|count|
+--------------+-----+
|           1.0|17173|
|           2.0|15177|
|          13.0|12903|
|           3.0|10891|
|           4.0| 4268|
|           5.0| 3966|
|           7.0| 2624|
|           6.0| 1021|
|          10.0|  612|
|           8.0|  160|
|          12.0|  106|
|           9.0|  100|
|          11.0|   40|
|           0.0|   10|
|         173.0|    1|
|         185.0|    1|
|         319.0|    1|
|         230.0|    1|
|         924.0|    1|
|          48.0|    1|
+--------------+-----+
only showing top 20 rows



Number of Reviews - AveragePrice

In [43]:
no_of_reviews_AvgPrice=My_df.groupBy("number_of_reviews").agg(func.mean("price").alias("AvgPrice")).orderBy("number_of_reviews",ascending=True)
no_of_reviews_AvgPrice.show()


+-----------------+-----------------+
|number_of_reviews|         AvgPrice|
+-----------------+-----------------+
|              0.0| 621.160101553896|
|              1.0| 627.795838511982|
|             10.0|635.7879867691727|
|            100.0|649.2588235294118|
|            101.0|664.7301587301587|
|           1010.0|           1097.0|
|            102.0|682.3023255813954|
|           1024.0|            121.0|
|            103.0|            600.4|
|            104.0|654.8036775698374|
|            105.0|556.6918271026665|
|            106.0|638.3846153846154|
|            107.0|673.9845862205364|
|            108.0|645.2179487179487|
|            109.0|671.4905660377359|
|             11.0|630.1375959310744|
|            110.0|597.3636363636364|
|            111.0|           621.32|
|            112.0|602.3157894736842|
|            113.0|            593.0|
+-----------------+-----------------+
only showing top 20 rows



NeighbourhoodGroup - Number of reviews

In [44]:
neighGroup_no_of_reviews= My_df.groupBy("neighbourhood_group").agg(func.mean("number_of_reviews").alias("no_of_reviews")).orderBy("no_of_reviews",ascending=False)
neighGroup_no_of_reviews.show()

+-------------------+------------------+
|neighbourhood_group|     no_of_reviews|
+-------------------+------------------+
|        unconfirmed|             168.5|
|           verified| 95.33333333333333|
|      Staten Island|37.371517027863774|
|             Queens|32.937167083378625|
|              Bronx| 31.41966173361522|
|           Brooklyn| 29.61399094111726|
|          Manhattan|24.498898043603567|
|              Nadia|              13.0|
|            Matthew|              10.0|
|           Katerine|               3.0|
|             Carmen|               2.0|
|      William Hakan|               2.0|
|             Evelyn|               2.0|
|             Krista|               2.0|
|             Sidney|               2.0|
|              Henry|               2.0|
|               Seth|               1.0|
|              Elien|               1.0|
|         Mott Haven|              null|
|                  2|              null|
+-------------------+------------------+
only showing top

Neighbourhood - Number of reviews

In [45]:
neighbourhood_no_of_reviews= My_df.groupBy("neighbourhood").agg(func.mean("number_of_reviews").alias("no_of_reviews")).orderBy("no_of_reviews",ascending=False)
neighbourhood_no_of_reviews.show()

+-------------------+------------------+
|      neighbourhood|     no_of_reviews|
+-------------------+------------------+
|          RoomPicks|             185.0|
|       Henry Street|             172.0|
|                Ben|             152.0|
|   East 47th Street|             136.0|
|               Greg|             133.0|
|           Huguenot|             110.5|
|           Avenue A|             109.0|
|       Richmondtown|             95.75|
|        Silver Lake|              94.0|
|      East Elmhurst| 78.89736070381231|
|        Eltingville| 67.33333333333333|
|    Manhattan Beach| 64.15384615384616|
|           Woodlawn| 61.64705882352941|
|Springfield Gardens|60.567708333333336|
|           Allerton| 59.09722222222222|
|            Clifton|59.035714285714285|
|              DUMBO|57.829787234042556|
|       New Brighton| 53.76923076923077|
|            Jamaica|  53.0551724137931|
|         St. George| 52.54651162790697|
+-------------------+------------------+
only showing top

Machine Learning model implementation

Seting up dataframe for ML

In [46]:
My_df.columns

['name',
 'host_id',
 'neighbourhood_group',
 'neighbourhood',
 'lat',
 'long',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [47]:
My_df.show()

+--------------------+-----------+-------------------+------------------+--------+---------+---------------+------+--------------+-----------------+-----------------+------------------------------+----------------+
|                name|    host_id|neighbourhood_group|     neighbourhood|     lat|     long|      room_type| price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+--------------------+-----------+-------------------+------------------+--------+---------+---------------+------+--------------+-----------------+-----------------+------------------------------+----------------+
|Clean & quiet apt...|80014485718|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room| 966.0|          10.0|              9.0|             0.21|                           6.0|           286.0|
|Skylit Midtown Ca...|52335172823|          Manhattan|           Midtown|40.75362|-73.98377|Entire home/apt| 142.0|          13.0|          

In [49]:
from pyspark.sql.types import IntegerType
My_df = My_df.withColumn("price",My_df["price"].cast(IntegerType()))


In [50]:
from pyspark.ml.feature import StringIndexer
My_df = StringIndexer(inputCol='neighbourhood_group', outputCol='neighbourhood_groupNEW').fit(My_df).transform(My_df)

My_df.show()

+--------------------+-----------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+----------------------+
|                name|    host_id|neighbourhood_group|     neighbourhood|     lat|     long|      room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|neighbourhood_groupNEW|
+--------------------+-----------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+----------------------+
|Clean & quiet apt...|80014485718|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  966|          10.0|              9.0|             0.21|                           6.0|           286.0|                   1.0|
|Skylit Midtown Ca...|52335172823|          Manhatta

In [51]:
from pyspark.ml.feature import StringIndexer
My_df = StringIndexer(inputCol='room_type', outputCol='room_typeNEW').fit(My_df).transform(My_df)

My_df.show()

+--------------------+-----------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+----------------------+------------+
|                name|    host_id|neighbourhood_group|     neighbourhood|     lat|     long|      room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|neighbourhood_groupNEW|room_typeNEW|
+--------------------+-----------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+----------------------+------------+
|Clean & quiet apt...|80014485718|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  966|          10.0|              9.0|             0.21|                           6.0|           286.0|                   1.0|         1.0|


In [52]:
My_df=My_df.na.fill(142,subset=['price'])

In [53]:
for col in My_df.columns:
  print(col, ":", My_df.where(My_df[col].isNull()).count())

name : 0
host_id : 0
neighbourhood_group : 0
neighbourhood : 0
lat : 0
long : 0
room_type : 0
price : 0
minimum_nights : 0
number_of_reviews : 0
reviews_per_month : 0
calculated_host_listings_count : 0
availability_365 : 0
neighbourhood_groupNEW : 0
room_typeNEW : 0


In [54]:
from pyspark.ml.feature import VectorAssembler

Asmblr=VectorAssembler(inputCols=["neighbourhood_groupNEW","room_typeNEW","price"],outputCol='VectorResult')

MYDF=Asmblr.transform(My_df)

MYDF.show()

+--------------------+-----------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+----------------------+------------+----------------+
|                name|    host_id|neighbourhood_group|     neighbourhood|     lat|     long|      room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|neighbourhood_groupNEW|room_typeNEW|    VectorResult|
+--------------------+-----------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+----------------------+------------+----------------+
|Clean & quiet apt...|80014485718|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  966|          10.0|              9.0|             0.21|                           6.0|   

K-Means Clustering

In [57]:

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

ClusterK = KMeans(featuresCol="VectorResult").setK(4).setSeed(1)
mod = ClusterK.fit(MYDF)
                   
pred = mod.transform(MYDF)

eval = ClusteringEvaluator(featuresCol="VectorResult")

silhouetteValue = eval.evaluate(pred)
print("Silhouette with squared euclidean distance = " + str(silhouetteValue))

ClusCenters = mod.clusterCenters()
print("Cluster Centers: ")
for CC in ClusCenters:
    print(CC)
                   

Silhouette with squared euclidean distance = 0.7308218755045838
Cluster Centers: 
[  0.79158565   0.48565199 476.09645866]
[8.04428044e-01 4.92137383e-01 1.05332739e+03]
[7.85918228e-01 4.87437186e-01 7.58044998e+02]
[  0.78769666   0.49148548 191.53779978]
